In [1]:
from Crypto.Hash import SHA256

In [2]:
BLOCK_SIZE = 1024;
SHA_SIZE   = SHA256.digest_size;

### reading a file

In [3]:
def read_raw_file(path):
    with open(path, "rb") as f:
        source = f.read()
    f.close()
    return source

### encoding and decoding a hashed message

In [4]:
def encode_hash(m: bytes):
    blocks  = []
    nfull   = len(m) // BLOCK_SIZE
    for i in range(nfull+1):
        i1 = BLOCK_SIZE * i
        i2 = min(BLOCK_SIZE * (i + 1), len(m))
        if i1 == i2: break
        blocks.append(m[i1:i2])
    hash_pad = b''
    for i in range(len(blocks)-1, -1, -1):
        block = blocks[i] + hash_pad
        sha = SHA256.new()
        sha.update(block)
        hash_pad = sha.digest()
        assert(len(hash_pad) == SHA_SIZE)
        blocks[i] = block
    return hash_pad.hex(), blocks

def decode_hash(h0, blocks):
    hash_pad = h0
    msg_blocks = []
    for j in range(len(blocks)-1):
        block = blocks[j]
        sha = SHA256.new()
        sha.update(block)
        block_hash = sha.digest()
        if len(block_hash) != len(hash_pad):
            return False, b''
        for i in range(len(hash_pad)):
            if block_hash[i] != hash_pad[i]:
                return False, b''
        hash_pad = block[-SHA_SIZE:]
        msg_blocks.append(block[0:-SHA_SIZE])
    msg_blocks.append(blocks[-1])
    m = b''.join(msg_blocks)
    return True, m
        

### tests

In [5]:
source = bytes('iau?!:".sdf/h,afu.837r3'*2000, 'ASCII')
h0, blocks = encode_hash(source)
ok, source1 = decode_hash(bytes.fromhex(h0), blocks)

print(ok)
assert(len(source) == len(source1))
for i in range(len(source)):
    assert(source[i] == source1[i])

True


In [6]:
source = read_raw_file('./6.2.birthday.mp4')
h0, blocks = encode_hash(source)
print(h0)
# validate
ok, source1 = decode_hash(bytes.fromhex(h0), blocks)
print(ok)
assert(len(source) == len(source1))
for i in range(len(source)):
    assert(source[i] == source1[i])

03c08f4ee0b576fe319338139c045c89c3e8e9409633bea29442e21425006ea8
True


In [7]:
source = read_raw_file('./6.1.intro.mp4')
h0, blocks = encode_hash(source)
print(h0)
# validate
ok, source1 = decode_hash(bytes.fromhex(h0), blocks)
print(ok)
assert(len(source) == len(source1))
for i in range(len(source)):
    assert(source[i] == source1[i])

5b96aece304a1422224f9a41b228416028f9ba26b0d1058f400200f06a589949
True
